# Problems

## Problem 1: Data from yfinance

The following function:

- downloads all hourly data for the previous five days for the five FAANG stocks.

   https://www.investopedia.com/terms/f/faang-stocks.asp

- saves the data into a folder called "data" in the root of this repository using a filename with the format "YYYYMMDD-HHmmss.csv" where "YYYYMMDD" is the four-digit year (e.g. 2025), followed by the two-digit month (e.g. 09 for September), followed by the two digit day, and "HHmmss" is hour, minutes, seconds. 

In [1]:
# Import required libraries.

# Dates and times.
import datetime as dt # For handling dates and times. 
# Data frames.
import pandas as pd # For data manipulation and analysis. 
# Yahoo Finance data.
import yfinance as yf # For downloading financial data from Yahoo Finance. 

import matplotlib.pyplot as plt 

from pathlib import Path


In [2]:
def get_data(days=5, interval='1h', folder='./data/'): 
    """
    This function downloads FAANG stock data and saves it to a CSV file with the following default parameters:
    - days: 5 (downloads data for the last 5 days)
    - interval: '1h' (hourly data)
    - folder: './data/' (saves the file in the 'data' folder in the root directory)
    The filename format is "YYYYMMDD-HHmmss.csv".
    """

    # Define the list of FAANG stocks.
    FAANG = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG']
    
    # Get the current date and time .
    now = dt.datetime.now()
    
    # Download hourly data for each FAANG stock.
    df = yf.download(FAANG, period=f'{days}d', interval=interval, auto_adjust=True)
    
    # Generate the filename with the current date and time.
    filename = folder + now.strftime("%Y%m%d-%H%M%S") + ".csv"
    
    # Save the data to a CSV file.
    df.to_csv(filename)

    return filename

In [3]:
get_data()

[*********************100%***********************]  5 of 5 completed

5 Failed downloads:
['NFLX', 'AAPL', 'GOOG', 'META', 'AMZN']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


'./data/20251120-001043.csv'

In [4]:
#get_data(days=3)

## Problem 2: Plotting Data

The following function:
- opens the latest data file in the "data" folder and, on one plot, plots the "Close" prices for each of the five stocks.

- saves the plot into the "plots" folder in the root of this repository using a filename in the format YYYYMMDD-HHmmss.png.


In [5]:
def plot_data(data_folder='./data/', plots_folder='./plots/'):
    """
    Load the most recent CSV from data_folder, plot Close prices for FAANG,
    show the plot in the notebook, and save it into plots_folder.
    """
    tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG']

    data_folder = Path(data_folder)
    plots_folder = Path(plots_folder)
    plots_folder.mkdir(parents=True, exist_ok=True)

    # Find latest CSV
    csv_files = sorted(data_folder.glob("*.csv"), key=lambda f: f.stat().st_mtime)
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the data folder.")
    latest_file = csv_files[-1]

    # Load yfinance multi-index CSV
    df = pd.read_csv(latest_file, header=[0,1], index_col=0, parse_dates=True)

    # Extract Close prices for available tickers
    available = [t for t in tickers if t in df['Close'].columns]
    close_df = df['Close'][available]

    # Ensure we have rows
    close_df = close_df.dropna(how='all')
    if close_df.shape[0] == 0:
        raise ValueError(
            "No usable price data found. "
            "Re-run get_data() with more days (e.g., days=10)."
        )

    # --- Create plot ---
    plt.figure(figsize=(12, 6))
    for t in available:
        plt.plot(close_df.index, close_df[t], label=t)

    plt.xlabel("Time")
    plt.ylabel("Close Price (USD)")
    plt.legend()

    title_date = pd.to_datetime(close_df.index[-1]).strftime("%Y-%m-%d %H:%M:%S")
    plt.title(f"FAANG Close Prices — {title_date}")

    # --- Save plot ---
    timestamp = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
    plot_path = plots_folder / f"{timestamp}.png"
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')

    # --- Show plot in notebook ---
    plt.show()

    # Close figure to free memory
    plt.close()

    return str(plot_path)

In [6]:
file = plot_data()
print("Saved to:", file)

ValueError: No usable price data found. Re-run get_data() with more days (e.g., days=10).

## Problem 3: Script

## Problem 4: Automation

## End